In [10]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [11]:
filename='230817_intent_labeled_by_6'

train_file = f'/home/azureuser/projects/aerius/ai/myapp/chatbot/datas/{filename}.csv'
data = pd.read_csv(train_file, delimiter = ',')
queries = data['query'].tolist()
intents = data['label'].tolist()

print(data.head(5))
print('len of queries = ', len(queries))
print('len of intents = ', len(intents))

                              query intent  label
0  아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?     주문    0.0
1             실수로 취소하면 재주문해야 하는 거죠?     주문    0.0
2                     택배비 따로 추가되나요?     배송    1.0
3                          택배비 있나요?     배송    1.0
4                        택배비 따로 들어요     배송    1.0
len of queries =  79617
len of intents =  79617


In [12]:
# train_data = data.sample(frac=0.7, random_state=42)
# temp_data = data.drop(train_data.index)
# val_data = temp_data.sample(frac=0.66, random_state=42)
# test_data = temp_data.drop(val_data.index)

# # Oversampling the minority classes in training data
# max_size = train_data['label'].value_counts().max()
# lst = [train_data]
# for class_index, group in train_data.groupby('label'):
#     lst.append(group.sample(max_size-len(group), replace=True))
# train_data_oversampled = pd.concat(lst)

# # Checking the distribution after oversampling
# oversampled_distribution = train_data_oversampled['label'].value_counts()

# oversampled_distribution

In [13]:
queries = data['query'].tolist()
intents = data['label'].tolist()

print(data.head(5))
print('len of queries = ', len(queries))
print('len of intents = ', len(intents))

                              query intent  label
0  아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?     주문    0.0
1             실수로 취소하면 재주문해야 하는 거죠?     주문    0.0
2                     택배비 따로 추가되나요?     배송    1.0
3                          택배비 있나요?     배송    1.0
4                        택배비 따로 들어요     배송    1.0
len of queries =  79617
len of intents =  79617


In [14]:
encoder = LabelEncoder()
encoder.fit(intents)

encoded_intents = encoder.transform(intents)
num_classes = len(encoder.classes_)
print(f"The number of unique labels is {num_classes}")

The number of unique labels is 6


In [15]:
# from ai.myapp.chatbot.utils.Preprocess import Preprocess

# # Preprocessing 객체 초기화
# p = Preprocess()

# # 첫 번째 문장 가져와서 처리
# sentence = queries[0]
# preprocessed = p.delete_intent_trash_tags(sentence=sentence)
# print("After delete_intent_trash_tags:", preprocessed)

# # 단어 리스트 생성
# words = []
# for sentence in queries:
#     if isinstance(sentence, str):  # Only process if the sentence is a string
#         preprocessed = p.delete_intent_trash_tags(sentence=sentence)
#         word_list, _ = p.divide_words_tags(preprocessed)
#         words.extend(word_list)
#     else:
#         print(f"Found non-string value: {sentence}")

# # words 리스트 확인
# print("Words List:", words[:30])  # 첫 30개의 단어만 출력
# print("Words List Length:", len(words))  # words 리스트의 길이 출력

# # 토크나이저 초기화
# p.initialize_tokenizer(words)

# # 토크나이저의 단어사전 확인
# print("Word Index:", p.tokenizer.word_index)

# # queries의 각 문장을 시퀀스로 변환
# sequences = []
# for sentence1 in queries:
#     sequence = p.text_to_sequence(sentence1)
#     sequences.append(sequence)
#     print("Original Sentence:", sentence1)
#     print("Converted Sequence:", sequence)

# # 변환된 시퀀스 확인
# print("Converted Sequences:", sequences[:20])  # 첫 20개의 시퀀스만 출력


In [16]:
from ai.myapp.chatbot.utils.Preprocess import Preprocess

# Step 1: Initialize Preprocessing object
p = Preprocess()

# Step 2: Preprocess the sentences
preprocessed_queries = []
for sentence in queries:
    if isinstance(sentence, str):  # Only process if the sentence is a string
        preprocessed = p.delete_intent_trash_tags(sentence=sentence)
        preprocessed_queries.append(preprocessed)
    else:
        print(f"Found non-string value: {sentence}")

# Print the preprocessed queries for debugging
print("Preprocessed Queries:")
print(preprocessed_queries[:5])

# Step 3: Extract words from the preprocessed queries
words = []
for preprocessed in preprocessed_queries:
    word_list, _ = p.divide_words_tags(preprocessed)
    words.extend(word_list)

# Print the extracted words for debugging
print("Extracted Words:")
print(words[:5])

# Step 4: Initialize the tokenizer with the extracted words
p.initialize_tokenizer(words)

# Step 5: Convert the preprocessed queries into sequences
sequences = []
for preprocessed in preprocessed_queries:
    word_list, _ = p.divide_words_tags(preprocessed)
    sentence = ' '.join(word_list)  # Join the words to form a sentence
    sequence = p.text_to_sequence(sentence)
    sequences.append(sequence)

# Print the converted sequences for debugging
print("Converted Sequences:")
print(sequences[:5])

# Print the tokenizer's word index for debugging
# print("Word Index:")
# print(p.tokenizer.word_index)


None
Preprocessed Queries:
[[('아침', 'NNG'), ('아니', 'VCN'), ('고', 'EC'), ('밤', 'NNG'), ('12', 'SN'), ('시', 'NNB'), ('30', 'SN'), ('분', 'NNB'), ('결제', 'NNG'), ('걸', 'VV'), ('ㄴ데', 'EC'), ('그렇', 'VA'), ('ㄴ가요', 'EF'), ('?', 'SF')], [('실수', 'NNG'), ('취소', 'NNG'), ('면', 'EC'), ('재', 'XPN'), ('주문', 'NNG'), ('아야', 'EC'), ('하', 'VV'), ('거', 'NNB'), ('죠', 'EF'), ('?', 'SF')], [('택배', 'NNP'), ('비', 'NNG'), ('따로', 'MAG'), ('추가', 'NNG'), ('나요', 'EF'), ('?', 'SF')], [('택배', 'NNP'), ('비', 'NNG'), ('있', 'VX'), ('나요', 'EF'), ('?', 'SF')], [('택배', 'NNP'), ('비', 'NNG'), ('따로', 'MAG'), ('들', 'VV'), ('어요', 'EC')]]
Extracted Words:
['아침', '아니', '고', '밤', '12']
Converted Sequences:
[[613, 71, 17, 681, 281, 13, 264, 399, 122, 30, 44, 271, 14], [682, 143, 28, 236, 19, 64, 11, 23, 26], [52, 91, 192, 258, 1], [52, 91, 4, 1], [52, 91, 192, 145, 24]]


In [17]:
print(words[:30])
print(sequences[:20])
print(sequence)


['아침', '아니', '고', '밤', '12', '시', '30', '분', '결제', '걸', 'ㄴ데', '그렇', 'ㄴ가요', '?', '실수', '취소', '면', '재', '주문', '아야', '하', '거', '죠', '?', '택배', '비', '따로', '추가', '나요', '?']
[[613, 71, 17, 681, 281, 13, 264, 399, 122, 30, 44, 271, 14], [682, 143, 28, 236, 19, 64, 11, 23, 26], [52, 91, 192, 258, 1], [52, 91, 4, 1], [52, 91, 192, 145, 24], [52, 91, 18, 1], [52, 91, 299, 111], [5, 8, 159, 4, 1], [15, 850, 112, 43, 111], [170, 5, 8, 2, 26], [5, 8, 91, 112], [21, 5, 8, 91, 111], [5, 8, 91, 1080, 101, 11, 1], [52, 91, 62, 29, 9, 1], [1030, 5, 8, 91, 112, 2, 14], [1602, 751, 175, 32, 25, 3, 5, 8, 91, 94, 299, 9, 1], [19, 29, 9, 28, 192, 5, 1241, 4, 1], [5, 8, 91, 112, 2, 463], [136, 13, 155, 2665, 80, 23, 2, 28, 860, 156, 271, 29, 140, 21, 16, 101, 11, 23, 71, 14], [1223, 179, 170, 5, 8, 2, 26]]
[7089, 40, 290, 10, 259, 3100, 1367, 583, 12870, 40, 347, 10, 259, 3100, 1367, 124, 15, 4, 46]


In [18]:
tokenizer_json = p.tokenizer.to_json()
with open(f'/home/azureuser/projects/aerius/ai/myapp/chatbot/models/tokenizers/{filename}_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)

In [19]:
print(sequence[:50])

[7089, 40, 290, 10, 259, 3100, 1367, 583, 12870, 40, 347, 10, 259, 3100, 1367, 124, 15, 4, 46]


In [20]:
from ai.backend.settings import INTENT_MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=INTENT_MAX_SEQ_LEN, padding='post')

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, encoded_intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.tokenizer.word_index) + 1

input_layer = Input(shape=(INTENT_MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=INTENT_MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters = 128,
               kernel_size = 3,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(filters = 128,
               kernel_size = 4,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(filters = 128,
               kernel_size = 5,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)


concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(num_classes, name='logits')(dropout_hidden)
predictions = Dense(num_classes, activation=tf.nn.softmax)(logits)

model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data = val_ds, epochs = EPOCH, verbose=1)

Epoch 1/5
2787/2787 [==============================] - 73s 25ms/step - loss: 0.4241 - accuracy: 0.8550 - val_loss: 0.2520 - val_accuracy: 0.9114
Epoch 2/5
2787/2787 [==============================] - 80s 29ms/step - loss: 0.2903 - accuracy: 0.9032 - val_loss: 0.2098 - val_accuracy: 0.9251
Epoch 3/5
2787/2787 [==============================] - 85s 31ms/step - loss: 0.2547 - accuracy: 0.9129 - val_loss: 0.1834 - val_accuracy: 0.9361
Epoch 4/5
2787/2787 [==============================] - 85s 31ms/step - loss: 0.2352 - accuracy: 0.9191 - val_loss: 0.1588 - val_accuracy: 0.9423
Epoch 5/5
2787/2787 [==============================] - 92s 33ms/step - loss: 0.2163 - accuracy: 0.9265 - val_loss: 0.1504 - val_accuracy: 0.9476


In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. 모델 예측
y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)

# test_ds_resampled에서 라벨만 추출
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# 2. 성능 지표 계산
precision = precision_score(y_true, y_pred_classes, average='weighted', zero_division=1)
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
accuracy = accuracy_score(y_true, y_pred_classes)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

399/399 [==============================] - 5s 6ms/step
Precision: 0.3027
Recall: 0.3075
F1 Score: 0.3050
Accuracy: 0.3075


In [22]:
loss, accuracy = model.evaluate(test_ds, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.15340444445610046
Test accuracy: 0.9452329874038696


In [23]:
loss, accuracy = model.evaluate(test_ds, verbose=1)
print(f'Accuracy: {accuracy * 100}')
print(f'loss: {loss}')

model.save(f'/home/azureuser/projects/aerius/ai/myapp/chatbot/models/{filename}_3.h5')

399/399 [==============================] - 3s 7ms/step - loss: 0.1416 - accuracy: 0.9509
Accuracy: 95.08855938911438
loss: 0.14161065220832825


/home/azureuser/projects/aerius/venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
